In [1]:
import pandas as pd

from research.plot import analyze_confidence_slices

df = pd.read_csv('model_l4_scores.csv')

analyze_confidence_slices(
    df['eob'],
    df['score'],
    df['volume'],
)

已生成: conf_0.98_0.99_ratio_0.1453.jpg (置信度区间: [0.98, 0.99], 样本占比: 14.53%)
已生成: conf_0.97_0.98_ratio_0.0969.jpg (置信度区间: [0.97, 0.98], 样本占比: 9.69%)
已生成: conf_0.96_0.97_ratio_0.0699.jpg (置信度区间: [0.96, 0.97], 样本占比: 6.99%)
已生成: conf_0.95_0.96_ratio_0.0443.jpg (置信度区间: [0.95, 0.96], 样本占比: 4.43%)
已生成: conf_0.94_0.95_ratio_0.0356.jpg (置信度区间: [0.94, 0.95], 样本占比: 3.56%)
已生成: conf_0.93_0.94_ratio_0.0253.jpg (置信度区间: [0.93, 0.94], 样本占比: 2.53%)
已生成: conf_0.92_0.93_ratio_0.0171.jpg (置信度区间: [0.92, 0.93], 样本占比: 1.71%)
已生成: conf_0.91_0.92_ratio_0.0199.jpg (置信度区间: [0.91, 0.92], 样本占比: 1.99%)
已生成: conf_0.90_0.91_ratio_0.0123.jpg (置信度区间: [0.90, 0.91], 样本占比: 1.23%)
已生成: conf_0.89_0.90_ratio_0.0117.jpg (置信度区间: [0.89, 0.90], 样本占比: 1.17%)
已生成: conf_0.88_0.89_ratio_0.0121.jpg (置信度区间: [0.88, 0.89], 样本占比: 1.21%)
已生成: conf_0.87_0.88_ratio_0.0121.jpg (置信度区间: [0.87, 0.88], 样本占比: 1.21%)
已生成: conf_0.86_0.87_ratio_0.0060.jpg (置信度区间: [0.86, 0.87], 样本占比: 0.60%)
已生成: conf_0.85_0.86_ratio_0.0073.jpg (置信度区间: [0.85, 0.86], 样本占比

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data_1 = pd.read_csv('model_l4_scores.csv')          ############   测试文件

data_1_size = data_1.shape[0]     ###### 测试数据行数  ###############
m_size = 25     ####### 测试多少个月 #######
buy = 1     ##### 多 ###################
sell = 0     ##### 空 ####################
rrr = 0.25     ###### 系数 ###################
m = 1000     ###### 总资金 ###################

In [2]:
ft = 0.01   ### 0.1, 0.05, 0.01

if ft == 0.1:
    da = 1
    xiao = 0.9
    z = 5

if ft == 0.05:
    da = 1
    xiao = 0.95
    z = 10
    
if ft == 0.01:
    da = 1
    xiao = 0.99
    z = 50


for i in range(z):        

    data_1['final'] = 0

    mask = (data_1['score'] < da) & (data_1['score'] >= xiao)
    data_1['final'] = np.where(mask, data_1['volume'], 0)
    
    true_count = mask.sum() / data_1_size 
    
    ### 更新置信度
    da = round((da - ft), 2)
    xiao = round((xiao - ft), 2)

    data_1['high'] = data_1['final'].cumsum()
    data_1['open'] = rrr * data_1['high'] / m

    fig=plt.figure(figsize=(20,10))  
    plt.plot(data_1['eob'], data_1['open'])
    plt.xticks(range(0,data_1_size,int(data_1_size/m_size)))   
    fig.autofmt_xdate()
    plt.grid(1)
    plt.savefig("./temp/" + str(da) + "_" + str(true_count.round(2)) + ".jpg")   
    plt.close()